In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np


In [ ]:
### Load the trained model,scaler pickle,onehot
model= load_model('model.h5')

## load the encoder and scaler
with open('lebel_encoder_gender.pkl','rb') as file:
    lebel_encoder_gender=pickle.load(file)

with open('onehot_encoder_Geograpy.pkl','rb') as file:
    onehot_encoder_Geograpy=pickle.load(file)

with open('scaler.pkl','rb') as file:
    scaler=pickle.load(file)



In [35]:
### Example input data

input_data = {
    "CreditScore": 720,
    "Geography": "Germany",         # Options: Germany, France, Spain
    "Gender": "Male",               # Options: Male, Female
    "Age": 35,
    "Tenure": 5,
    "Balance": 75000.00,
    "NumOfProducts": 2,
    "HasCrCard": 1,                 # 1 if the customer has a credit card, else 0
    "IsActiveMember": 1,            # 1 if the customer is active, else 0
    "EstimatedSalary": 85000.00
}


In [44]:
input_df=pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,720,Germany,1,35,5,75000.0,2,1,1,85000.0


In [ ]:
input_data['Gender']=lebel_encoder_gender.transform([input_data['Gender']])[0]

In [45]:
geo_encoded = onehot_encoder_Geograpy.transform([[input_data['Geography']]])

c:\Users\rajap\OneDrive\Desktop\Deep Learning\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


In [46]:
geo_encoded_df=pd.DataFrame(geo_encoded,columns=onehot_encoder_Geograpy.get_feature_names_out(['Geography']))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,0.0,1.0,0.0


In [50]:
input_df=pd.concat([input_df.drop('Geography',axis=1),geo_encoded_df],axis=1)

In [51]:
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,720,1,35,5,75000.0,2,1,1,85000.0,0.0,1.0,0.0


In [54]:
### scaler
scaled_df=scaler.transform(input_df)
scaled_df

array([[ 7.09342564e-01,  9.13247552e-01, -3.70568589e-01,
        -1.34471949e-03, -1.76463463e-02,  8.08436154e-01,
         6.49202671e-01,  9.74816989e-01, -2.68298748e-01,
        -9.98501123e-01,  1.72572313e+00, -5.76388018e-01]])

In [56]:
prediction=model.predict(scaled_df)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step


In [57]:
probability=prediction[0][0]

In [58]:
probability

np.float32(0.029964363)

In [59]:
if probability>0.5:
    print("churn")
else:
    print("not churn")

not churn
